<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библитек-и-модулей" data-toc-modified-id="Импорт-библитек-и-модулей-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библитек и модулей</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

### Импорт библитек и модулей 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import math

from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import  make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer, mean_absolute_error, accuracy_score, r2_score, f1_score, recall_score, precision_score, roc_auc_score,roc_curve, confusion_matrix

import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.info())
display(df.describe())
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — обратимая матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**  Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится

**Обоснование:** пусть $w1$ вектор весов линейной регрессии матрицы $XP$, а $a1$ предсказания матрицы $XP$ тогда:


$$
w1 = ((XP^T) (XP))^{-1} (XP)^T y
$$
- Раскрываем скобки
$$
w1 = (P^T (X^TX))^{-1} (XP)^T y
$$

$$
w1 = (P^T X^T(XP))^{-1} (XP)^T y
$$

$$
w1 = (P^T(X^T X)P)^{-1} (XP)^T y
$$ 

$$
w1 = P^{-1}(X^T X)^{-1} (P^T)^{-1}  P^T X^T y
$$ 

$$
w1 = P^{-1}(X^T X)^{-1} [(P^T)^{-1} P^T] X^T y
$$
 - $[(P^T)^{-1} P^T] = E$ , любая матрица умноженная на едеиничную матрицу равна самой себе следовательно,
$$
w1 = P^{-1}[(X^T X)^{-1}  X^T y]
$$

$$
w1 = P^{-1}w
$$

$$
a1 = X [P P^{-1}]w
$$

$$
a1 = Xw
$$

$$
a1 = a
$$

Предсказания не изменятся если матрицу умножить на обратимую матрицу, следовательно качество линейной регрессии не поменяется

## Алгоритм преобразования

Для защиты информации будем умножать матрицу признаков на случайно сгенерированную обратимую матрицу

**Алгоритм**

1. Создаем случайную матрицу
2. Проверяем на обратимость
3. Преобразуем признаки 
4. Обучаем модель

**Обоснование**
Т. к мы в пункте 2 доказали, что умножение на обратимую матрицу не меняет качество линейной регресии, будем персональные данные умножать на случайную матрицу размером   nxn где n = число переменных признаков 

## Проверка алгоритма

In [3]:
features = df.drop("Страховые выплаты", axis=1)#переменные признаки
target = df["Страховые выплаты"] # целевой признак

In [4]:
def r2(data):  #функция вычесления r2
    features = df.drop("Страховые выплаты", axis=1)
    target = df["Страховые выплаты"]
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)
    print(r2_score(target, predictions))
    

In [5]:
r2(df)

0.42494550286668


In [6]:
P = np.linalg.inv(np.random.sample((4, 4)))

In [7]:
new_df =  features @ P
display(new_df.head(10))#проверка результата, признаки преобразовались в нечто новое

,0,1,2,3
0,-26218.549108,-61107.314143,147724.906328,107436.298181
1,-20067.851480,-46816.854230,113170.536716,82288.133481
2,-11086.316013,-25873.816812,62544.064926,45470.697651
3,-22056.571140,-51370.438321,124186.719358,90340.411581
4,-13789.876877,-32157.198320,77739.034606,56526.506735
5,-21660.778799,-50511.264253,122106.364871,88794.411989
6,-20977.143469,-48909.299650,118234.678233,85982.633267
7,-20409.660208,-47549.191725,114949.665733,83615.824420
8,-26277.541082,-61229.935209,148022.942175,107659.989104
9,-27341.540099,-63693.334956,153979.792163,111999.959525


In [8]:
r2(new_df)

0.42494550286668


Качество линейной регресии на преобразованный признаках не поменялось, что говорит нам Алгоритм работает

In [12]:
if r2(df) == r2(new_df):
    print('збс, все работает!!!')
else:
    print('всё гавно, переделывай!!!')

0.42494550286668
0.42494550286668
збс, все работает!!!
